In [1]:
import mpp_classifiers as mpp
import load_ship_data as lsd
import numpy as np
import time
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold

Get data using our own data loader

NOTE: Confusion Matrix is incorrect. 

I corrected it in reported results, but left it this way in code

so that results would line up with report.

Rerunning with correct matrix would cause our random split to alter the results slightly

In [2]:

data_path="data/shipsnet.json"
train,test,valid=lsd.load_data_train_test_split(data_path)
def conf(pred,y):
    T0=sum([1 if x==y and y==0 else 0 for (x,y) in zip(pred,y)])
    T1=sum([1 if x==y and y==1 else 0 for (x,y) in zip(pred,y)])
    F0=sum([1 if x!=y and y==0 else 0 for (x,y) in zip(pred,y)])
    F1=sum([1 if x!=y and y==1 else 0 for (x,y) in zip(pred,y)])
    return np.array([[T0,T1],[F0,F1]])

Training Set Data Length:  2800   Label Length:  2800
TestingSet Set Data Length:  600  Label Length:  600
Validation Set Data Length:  600  Label Length:  600


Setup data splits and priors

In [3]:
ship_prob=np.sum(train[1])*1.0/train[1].shape[0]
p=np.array([1-ship_prob,ship_prob])
print(p)
Xtrain=train[0]
ytrain=train[1]
Xtest=test[0]
ytest=test[1]
Xvalid=valid[0]
yvalid=valid[1]
Xtrain=Xtrain.reshape(Xtrain.shape[0],Xtrain.shape[1]*Xtrain.shape[2]*Xtrain.shape[3])
Xtest=Xtest.reshape(Xtest.shape[0],Xtest.shape[1]*Xtest.shape[2]*Xtest.shape[3])
Xvalid=Xvalid.reshape(Xvalid.shape[0],Xvalid.shape[1]*Xvalid.shape[2]*Xvalid.shape[3])
print(Xtrain.shape)
print(Xtest.shape)
print(Xvalid.shape)

[0.75035714 0.24964286]
(2800, 19200)
(600, 19200)
(600, 19200)


Case 1. No PCA, even priors

In [4]:

p55=np.array([.5,.5])
t0 = time.time()           # start time
y_model = mpp.mpp(Xtrain, ytrain, Xtest, 1, p55)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print(ar)

Overall accuracy = 0.6633333333333333;
Classwise accuracy = [0.65759637 0.67924528];
The learning process takes 14.288885354995728 seconds.
[[290 108]
 [151  51]]


Case 2. No PCA, prior assumes ships are rare

In [5]:
p91=np.array([.9,.1])
t0 = time.time()           # start time
y_model = mpp.mpp(Xtrain, ytrain, Xtest, 1, p91)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print(ar)

Overall accuracy = 0.735;
Classwise accuracy = [1. 0.];
The learning process takes 14.406003713607788 seconds.
[[441   0]
 [  0 159]]


Case 1. No PCA, priors from training data

In [6]:

t0 = time.time()           # start time
y_model = mpp.mpp(Xtrain, ytrain, Xtest, 1, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print(ar)

Overall accuracy = 0.7366666666666667;
Classwise accuracy = [1.         0.00628931];
The learning process takes 14.272328853607178 seconds.
[[441   1]
 [  0 158]]


PCA = 800 dimensions kept

In [7]:
pca = PCA(n_components=800)
pca.fit(Xtrain)
pca_Xtrain=pca.transform(Xtrain)
pca_Xtest=pca.transform(Xtest)
explained=np.sum(pca.explained_variance_ratio_)
print(explained)

0.9906665778462846


Case 1. PCA=800, training priors used from now on

In [8]:
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(pca_Xtrain, ytrain, pca_Xtest, 1, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print(ar)

Overall accuracy = 0.7366666666666667;
Classwise accuracy = [1.         0.00628931];
The learning process takes 0.08228778839111328 seconds.
[[441   1]
 [  0 158]]


Case 2. PCA=800

In [9]:
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(pca_Xtrain, ytrain, pca_Xtest, 2, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print(ar)

Overall accuracy = 0.9016666666666666;
Classwise accuracy = [0.92290249 0.8427673 ];
The learning process takes 24.81798505783081 seconds.
[[407 134]
 [ 34  25]]


Case 3. PCA=800

In [10]:
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(pca_Xtrain, ytrain, pca_Xtest, 3, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print(ar)

Overall accuracy = 0.5083333333333333;
Classwise accuracy = [0.52380952 0.46540881];
The learning process takes 31.980409622192383 seconds.
[[231  74]
 [210  85]]


PCA =200 dimensions kept

In [11]:
pca2 = PCA(n_components=200)
pca2.fit(Xtrain)
pca2_Xtrain=pca2.transform(Xtrain)
pca2_Xtest=pca2.transform(Xtest)
explained=np.sum(pca2.explained_variance_ratio_)
print(explained)

0.9425671638421523


Case 1. PCA=200

In [12]:
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(pca2_Xtrain, ytrain, pca2_Xtest, 1, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print(ar)

Overall accuracy = 0.74;
Classwise accuracy = [1.         0.01886792];
The learning process takes 0.024442195892333984 seconds.
[[441   3]
 [  0 156]]


Case 2. PCA=200

In [13]:
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(pca2_Xtrain, ytrain, pca2_Xtest, 2, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print(ar)

Overall accuracy = 0.9233333333333333;
Classwise accuracy = [0.96145125 0.81761006];
The learning process takes 0.569476842880249 seconds.
[[424 130]
 [ 17  29]]


Case 3. PCA=200

In [14]:
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(pca2_Xtrain, ytrain, pca2_Xtest, 3, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print(ar)

Overall accuracy = 0.8283333333333334;
Classwise accuracy = [0.81632653 0.86163522];
The learning process takes 0.9020977020263672 seconds.
[[360 137]
 [ 81  22]]


Join training and validation since upcoming cross-validation will do this split for us

In [15]:
pca2 = PCA(n_components=200)
Xtrain=np.concatenate((Xtrain,Xvalid))
ytrain=np.concatenate((ytrain,yvalid))
pca2.fit(Xtrain)
pca2_Xtrain=pca2.transform(Xtrain)
pca2_Xtest=pca2.transform(Xtest)
explained=np.sum(pca2.explained_variance_ratio_)
print(explained)

0.9391647331330492


Setup for Kfold cross-validation

In [16]:
p=np.array([1-ship_prob,ship_prob])
X=pca2_Xtrain
y=ytrain
k=10

Case 1

In [17]:

kf=KFold(n_splits=k,random_state=None)
acc_scores=[]
for train_index,test_index in kf.split(X):
    X_train,X_test=X[train_index,:],X[test_index,:]
    y_train,y_test=y[train_index],y[test_index]
    y_model = mpp.mpp(X_train, y_train, X_test, 1, p)
    acc_classwise, acc_overall = mpp.accuracy_score(y_test, y_model)
    acc_scores.append(acc_overall)
avg_acc=sum(acc_scores)*1.0/k
print('Accuracy from each fold =  {}'.format(acc_scores))
print('Average Accuracy = {}'.format(avg_acc))
y_model = mpp.mpp(X_train, y_train, X_test, 1, p)
acc_classwise, acc_overall = mpp.accuracy_score(y_test, y_model)
print(acc_overall)

Accuracy from each fold =  [0.7176470588235294, 0.7470588235294118, 0.7294117647058823, 0.7529411764705882, 0.7558823529411764, 0.7676470588235295, 0.7558823529411764, 0.788235294117647, 0.75, 0.7735294117647059]
Average Accuracy = 0.7538235294117647
0.7735294117647059


Case 2

In [18]:
kf=KFold(n_splits=k,random_state=None)
acc_scores=[]
for train_index,test_index in kf.split(X):
    X_train,X_test=X[train_index,:],X[test_index,:]
    y_train,y_test=y[train_index],y[test_index]
    y_model = mpp.mpp(X_train, y_train, X_test, 2, p)
    acc_classwise, acc_overall = mpp.accuracy_score(y_test, y_model)
    acc_scores.append(acc_overall)
avg_acc=sum(acc_scores)*1.0/k
print('Accuracy from each fold =  {}'.format(acc_scores))
print('Average Accuracy = {}'.format(avg_acc))
y_model = mpp.mpp(X_train, y_train, X_test, 2, p)
acc_classwise, acc_overall = mpp.accuracy_score(y_test, y_model)
print(acc_overall)



Accuracy from each fold =  [0.9176470588235294, 0.9088235294117647, 0.9176470588235294, 0.9411764705882353, 0.9235294117647059, 0.9264705882352942, 0.9264705882352942, 0.9294117647058824, 0.9264705882352942, 0.9058823529411765]
Average Accuracy = 0.9223529411764707
0.9058823529411765


Case 3

In [19]:
kf=KFold(n_splits=k,random_state=None)
acc_scores=[]
for train_index,test_index in kf.split(X):
    X_train,X_test=X[train_index,:],X[test_index,:]
    y_train,y_test=y[train_index],y[test_index]
    y_model = mpp.mpp(X_train, y_train, X_test, 3, p)
    acc_classwise, acc_overall = mpp.accuracy_score(y_test, y_model)
    acc_scores.append(acc_overall)
avg_acc=sum(acc_scores)*1.0/k
print('Accuracy from each fold =  {}'.format(acc_scores))
print('Average Accuracy = {}'.format(avg_acc))
y_model = mpp.mpp(X_train, y_train, X_test, 3, p)
acc_classwise, acc_overall = mpp.accuracy_score(y_test, y_model)
print(acc_overall)



Accuracy from each fold =  [0.7941176470588235, 0.7441176470588236, 0.8058823529411765, 0.7617647058823529, 0.8352941176470589, 0.788235294117647, 0.7852941176470588, 0.788235294117647, 0.8058823529411765, 0.8235294117647058]
Average Accuracy = 0.793235294117647
0.8235294117647058
